In [18]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os # for creating directories

In [19]:
env = gym.make('MountainCar-v0') # initialise environment
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
batch_size = 32
n_episodes = 1001 # n games we want agent to play (default 1001)
output_dir = 'model_output/mountaincar/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [38]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) 
        self.epsilon = 1.0
        self.model = self._build_model() 
    
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu')) 
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear')) 
        model.compile(loss='mse',optimizer=Adam(lr=0.001))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
        
    def act(self, state):
        
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            act_values = self.model.predict(state) 
        
        return np.argmax(act_values[0])

    def replay(self, batch_size): 
        
        minibatch = random.sample(self.memory, batch_size) 
#         minibatch = self.memory 
        
        for state, action, reward, next_state, done in minibatch: 
        
            if done:
                target = reward
            else: 
                target = reward + 0.97 * np.amax(self.model.predict(next_state)[0])
            
            target_f = self.model.predict(state)
            target_f[0][action] = target
        
            self.model.fit(state, target_f, epochs=1, verbose=0)
        
#         if self.epsilon > 0.01:
#             self.epsilon = self.epsilon * 0.995

In [44]:
agent = DQNAgent(state_size, action_size) # initialise agent
## done = False
successes = 0
for e in range(n_episodes): 
    state = env.reset()
    s1 = state
    state = np.reshape(state, [1, state_size])

    for time in range(500): 
#         env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        
        
        ns1 = next_state
        reward = next_state[0] + 0.5
        if next_state[0] >= 0.5:
            reward += 1
            
        next_state = np.reshape(next_state, [1, state_size])
       
        state = next_state   
        
        target = reward
        target_f = agent.model.predict(state)
        target_f[0][action] = target
        agent.model.fit(state, target_f, epochs=1, verbose=0)
        
        if done:
            if  ns1[0] >= 0.5:
                if agent.epsilon > 0.01:
                    agent.epsilon *= .995
#             successes += 1

           
            print("episode: {}/{}, score: {:.2}, e: {:.2}".format(e, n_episodes, ns1[0], agent.epsilon))
            break

episode: 0/1001, score: -0.58, e: 1.0
episode: 1/1001, score: -0.62, e: 1.0
episode: 2/1001, score: -0.53, e: 1.0
episode: 3/1001, score: -0.46, e: 1.0
episode: 4/1001, score: -0.39, e: 1.0
episode: 5/1001, score: -0.52, e: 1.0
episode: 6/1001, score: -0.64, e: 1.0
episode: 7/1001, score: -0.55, e: 1.0
episode: 8/1001, score: -0.68, e: 1.0
episode: 9/1001, score: -0.51, e: 1.0
episode: 10/1001, score: -0.53, e: 1.0
episode: 11/1001, score: -0.59, e: 1.0
episode: 12/1001, score: -0.57, e: 1.0
episode: 13/1001, score: -0.4, e: 1.0
episode: 14/1001, score: -0.3, e: 1.0
episode: 15/1001, score: -0.54, e: 1.0
episode: 16/1001, score: -0.51, e: 1.0
episode: 17/1001, score: -0.57, e: 1.0
episode: 18/1001, score: -0.56, e: 1.0
episode: 19/1001, score: -0.43, e: 1.0
episode: 20/1001, score: -0.47, e: 1.0
episode: 21/1001, score: -0.45, e: 1.0
episode: 22/1001, score: -0.63, e: 1.0
episode: 23/1001, score: -0.51, e: 1.0
episode: 24/1001, score: -0.63, e: 1.0
episode: 25/1001, score: -0.59, e: 1.

KeyboardInterrupt: 